In [37]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
from shapely.geometry import Polygon

In [38]:
## Chargement des données de références :
os.chdir('c:/Users/alexa/Bureau/Data_challenge/') 
## Pour colab :
'''
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/data_challenge_mines/')
'''

ref = pd.read_csv("datasets_train/train_annotation/_annotation.csv")
## Nettoyage des données 
ref = ref[ np.logical_and(ref['class'] == 1, ref['models'].notna()) ].set_index('im_name')


In [39]:
# métrique IoU recomandée 
def local_IoU(
    xmin_pred_i,
    xmax_pred_i,
    ymin_pred_i,
    ymax_pred_i,
    xmin_true_i,
    xmax_true_i,
    ymin_true_i,
    ymax_true_i,
):
    """This function calculates the IoU for the image i.

    Args:
        xmin_pred_i: Value of the prediction min x-axis.
        xmax_pred_i: Value of the prediction max x-axis.
        ymin_pred_i: Value of the prediction min y-axis.
        ymax_pred_i: Value of the prediction max y-axis.
        xmin_true_i: Value of the true min x-axis.
        xmax_true_i: Value of the true max x-axis.
        ymin_true_i: Value of the true min y-axis.
        ymax_true_i: Value of the true max y-axis.

    Returns:
        The return value is the intersection over union.

    """
    if (xmin_true_i, xmax_true_i, ymin_true_i, ymax_true_i) == (0, 0, 0, 0):
        if (xmin_pred_i, xmax_pred_i, ymin_pred_i, ymax_pred_i) == (
            0,
            0,
            0,
            0,
        ):
            return 1

        else:
            return 0

    else:
        box_pred_i = [
            [xmin_pred_i, ymin_pred_i],
            [xmax_pred_i, ymin_pred_i],
            [xmax_pred_i, ymax_pred_i],
            [xmin_pred_i, ymax_pred_i],
        ]
        box_true_i = [
            [xmin_true_i, ymin_true_i],
            [xmax_true_i, ymin_true_i],
            [xmax_true_i, ymax_true_i],
            [xmin_true_i, ymax_true_i],
        ]
        poly_1 = Polygon(box_pred_i)
        poly_2 = Polygon(box_true_i)
        try:
            iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
            return iou
        except:
            return 0


In [40]:
def test_unit( model , image_name):
    image = cv2.imread(image_name)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_tensor = tf.convert_to_tensor(image, dtype = tf.uint8)
    image_tensor = tf.expand_dims(image_tensor, axis = 0)
    detection = model(image_tensor)
    box = (detection['detection_boxes'][0][0])
    y1, x1, y2, x2 = box #le y avant le x pk pas 
    (x1, y1) = (int(x1 * image.shape[1]), int(y1 * image.shape[0])) #coordonées absolues
    (x2, y2) = (int(x2 * image.shape[1]), int(y2 * image.shape[0])) 
    # calcul de la performance :
    (X1, Y1, X2, Y2) = ref.loc[image_name][['x_min', 'y_min', 'x_max', 'y_max']]
    IoU = local_IoU(x1, x2, y1, y2, X1, X2, Y1, Y2)
    return IoU
    

In [5]:

FasterRCNN = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1")

RetinaNet = hub.load("https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1")

SSD_mobilenet_320 = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

SSD_mobilenet_640 = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1")

In [41]:
def test(model, image_list):
    l = []
    t1 = time.time()
    for image_name in image_list:
        IoU = test_unit(model, image_name)
        l.append(IoU)
    l = np.array(l)
    error = np.sum(l)
    t2 = time.time()
    return (t2 - t1), error

In [42]:
'''## En cas d'éxécution sur Colab :
from google.colab import drive
drive.mount('/content/drive')
image_path = "/content/drive/MyDrive/data_challenge_mines/datasets_train/train/" #chemin d'accès
os.chdir(image_path)
'''

## En cas d'éxécution en local :
image_path = "datasets_train/train/" #chemin d'accès 
os.chdir(image_path)


In [43]:
# image_list = np.array( ref.index) pour tester sur tout le dataset (très long, à faire une seule fois)
image_list = np.array( ref.index)[:10] # version toy pour tester

delta_t1, IoU1 = test(FasterRCNN, image_list)
delta_t2, IoU2 = test(RetinaNet, image_list)
delta_t3, IoU3 = test(SSD_mobilenet_320, image_list)
delta_t4, IoU4 = test(SSD_mobilenet_640, image_list)

time_delta = [delta_t1, delta_t2, delta_t3, delta_t4]
IoU = [IoU1, IoU2, IoU3, IoU4]
name = ['FasterRCNN', 'RetinaNet', 'SSD_mobilenet_320', 'SSD_mobilenet_640']

benchmark = pd.DataFrame({'Method' : name , 'IoU' : IoU, 'time_delta' : time_delta})
os.chdir('c:/Users/alexa/Bureau/Data_challenge/')
# Ou pour colab :
#os.chdir('/content/drive/MyDrive/data_challenge_mines/')
benchmark.to_csv('benchmarck.csv')
